In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures

from fibsem.structures import BeamType, Point, FibsemRectangle


In [ ]:
microscope, settings = utils.setup_session(config_path=r"C:\Users\Admin\Github\autolamella\autolamella")

In [ ]:

settings.image.beam_type = BeamType.ION
ref_image = acquire.new_image(microscope, settings.image)


In [ ]:




pixelsize = ref_image.metadata.pixel_size.x
fiducial_centre = Point(-50e-6, -10e-6)
fiducial_length = 30e-6 # microscope_settings.protocol["fiducial"]["length"] 
fiducial_area = calculate_fiducial_area(settings, fiducial_centre, fiducial_length, pixelsize)

print(fiducial_area)

settings.image.reduced_area = fiducial_area
image = acquire.new_image(microscope, settings.image)
print(image.metadata.image_settings.reduced_area)


## Minimap
Position Display

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures
from fibsem.structures import FibsemImage, BeamType
import os
from fibsem.imaging import _tile

import matplotlib.pyplot as plt
from autolamella.structures import Experiment

# microscope, settings = utils.setup_session()


In [ ]:

PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\TEST_DEV_01"
# EXP_PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-26072023\experiment.yaml"
exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
# image = FibsemImage.load(os.path.join(PATH, "overview-image-ion-auto-gamma.tif"))
ion_image = FibsemImage.load(os.path.join(PATH, "overview-ion-start.tif"))
# ion_image = FibsemImage.load(os.path.join(PATH, "overview-ion-trench.tif"))

plt.imshow(ion_image.data, cmap="gray")
plt.show()

In [ ]:
from autolamella.tools.data import create_history_dataframe, calculate_statistics_dataframe
import plotly.express as px
import pandas as pd


pd.set_option('display.max_rows', 300)

df_experiment, df_history, df_beam_shift, df_steps, df_stage = calculate_statistics_dataframe(PATH)

# display(df_steps)


In [ ]:
fig = px.bar(df_history, x="stage", y="duration", color="petname", barmode="group", hover_data=df.columns)
fig.show()

In [ ]:
# display(df_stage)

df_stage_filter = df_stage[df_stage["stage"] != "Setup"]
# drop relative moves 
df_stage_filter = df_stage_filter[df_stage_filter["type"] != "relative"]

# drop x=0, y=0
df_stage_filter = df_stage_filter[df_stage_filter["x"] != 0]
df_stage_filter = df_stage_filter[df_stage_filter["y"] != 0]


px.line(df_stage_filter, x="x", y="y", color="stage",symbol="lamella",  title="Stage History")


In [ ]:
# display(df_steps)

fig = px.bar(df_steps, x="lamella", y="duration", color="step", title="Step Duration", barmode="stack", facet_col="stage")

fig.show()
# plot time series with x= step_n and y = timestamp with step  as hover text
df_steps.dropna(inplace=True)
df_steps.duration =df_steps.duration.astype(int)

# convert timestamp to datetime, aus timezone 
df_steps.timestamp = pd.to_datetime(df_steps.timestamp, unit="s")

# convert timestamp to australian timezone
df_steps.timestamp = df_steps.timestamp.dt.tz_localize("UTC").dt.tz_convert("Australia/Sydney")

fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol="lamella",
    title="AutoLamella Timeline", 
    hover_name="stage", hover_data=["lamella", "step_n", "step"],)
    # size = "duration", size_max=20)

fig.show()

In [ ]:

from autolamella.tools.data import create_history_dataframe
df = create_history_dataframe(exp)
display(df)

import plotly.express as px

fig = px.bar(df, x="petname", y="duration", color="stage", barmode="group", hover_data=df.columns)
fig.show()

In [ ]:

positions = {
    "MillTrench": [],
    "MillUndercut": [],
    "ReadyLamella": [],
}
for lamella in exp.positions:
    for state in lamella.history:
        
        if state.stage.name in positions.keys():
            positions[state.stage.name].append(state.microscope_state.absolute_position)
            positions[state.stage.name][-1].name = f"{lamella._petname}"# [{state.stage.name}]"

# pprint(positions)

In [ ]:

key = "MillTrench"
fig = _tile._plot_positions(ion_image, positions[key], show=True)
fig.savefig(os.path.join(PATH, f"overview-positions-{key.lower()}.png"), dpi=300, bbox_inches='tight')

In [ ]:
fig = _tile._plot_positions(eb_image, positions["MillUndercut"], show=True)
fig.savefig(os.path.join(PATH, "overview-positions-mill-undercut.png"), dpi=300, bbox_inches='tight')


In [ ]:
fig = _tile._plot_positions(eb_image, positions["ReadyLamella"], show=True)
fig.savefig(os.path.join(PATH, "overview-positions-mill-lamella.png"), dpi=300, bbox_inches='tight')

In [ ]:
_n

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils, acquire
from fibsem.structures import FibsemRectangle, MicroscopeState, Point, BeamType, FibsemImage
from autolamella import waffle as wfl
from autolamella.structures import Experiment, Lamella
import os
import glob

from fibsem.imaging import _tile
from pprint import pprint 


In [ ]:
PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-02-02082023"

TILE_PATH = "overview-ion-grid01"



filenames0 = sorted(glob.glob(os.path.join(PATH, TILE_PATH, "tile_0*.tif")))
filenames1 = sorted(glob.glob(os.path.join(PATH, TILE_PATH, "tile_1*.tif")))
filenames2 = sorted(glob.glob(os.path.join(PATH, TILE_PATH, "tile_2*.tif")))
filenames3 = sorted(glob.glob(os.path.join(PATH, TILE_PATH, "tile_3*.tif")))



images= [[FibsemImage.load(fname) for fname in filenames0], 
         [FibsemImage.load(fname) for fname in filenames1],
            [FibsemImage.load(fname) for fname in filenames2],
            [FibsemImage.load(fname) for fname in filenames3]]
# big_image = FibsemImage.load(os.path.join(PATH, TILE_PATH, f"big_image_ib.tif"))



# plot all images as a grid

fig, ax = plt.subplots(4,4, figsize=(10,10))

for i in range(4):
    for j in range(4):
        ax[i,j].imshow(images[i][j].data, cmap="gray")
        ax[i,j].axis("off")

plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

# ddict = utils.load_yaml(os.path.join(PATH, TILE_PATH, f"{TILE_PATH}.yaml"))
# ddict["images"] = images
# ddict["big_image"] = big_image
# ddict["prev-label"] = "overivew-ion-grid02-clahe"

# from fibsem.structures import ImageSettings

# ddict["image_settings"] = ImageSettings.__from_dict__(ddict["image_settings"])
# ddict["start_state"] = MicroscopeState.__from_dict__(ddict["start_state"])
# pprint(ddict)

In [ ]:
image = _tile._stitch_images(ddict["images"], ddict, overlap=0)


tile_image = FibsemImage.load(os.path.join(PATH, f"{TILE_PATH}.tif"))

# plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(30, 15))
ax[0].imshow(image.data, cmap="gray")
ax[1].imshow(tile_image.data, cmap="gray")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# create a grid of lines on a blank image

grid_shape = (4*1024, 4*1024)
arr = np.zeros(shape=grid_shape, dtype=np.uint8)


# create grid, grid bars thickness = 10px
BAR_THICKNESS_PX = 50
BAR_SPACING_PX = 170
for i in range(0, arr.shape[0], BAR_SPACING_PX ):
    arr[i:i+BAR_THICKNESS_PX, :] = 255
    arr[:, i:i+BAR_THICKNESS_PX] = 255

import tifffile as tff
PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-02-02082023"

tff.imwrite(os.path.join(PATH, "grid-bar.tif"), arr)

fig = plt.figure(figsize=(15, 15))
plt.imshow(arr, cmap="gray")
plt.axis("off")
plt.show()


### DETECTION LOGGING RE-IMPLEMENTATION

In [1]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures
from fibsem.structures import FibsemImage, BeamType
import os
from fibsem.imaging import _tile

import matplotlib.pyplot as plt
from autolamella.structures import Experiment


from autolamella.tools.data import create_history_dataframe, calculate_statistics_dataframe
import plotly.express as px
import pandas as pd


pd.set_option('display.max_rows', 300)

PATH = "/home/patrick/github/autolamella/autolamella/log/DEV_DET_LOGGING"
df_experiment, df_history, df_beam_shift, df_steps, df_stage, df_det = calculate_statistics_dataframe(PATH)


display(df_experiment)


--------------------------------------------------------------------------------


,experiment_name,experiment_path,experiment_created_at,experiment_id,number,petname,path,lamella.x,lamella.y,lamella.z,lamella.r,lamella.t,last_timestamp,name
0,DEV_DET_LOGGING,/home/patrick/github/autolamella/autolamella/l...,1.691405e+09,None,1,01-nice-finch,/home/patrick/github/autolamella/autolamella/l...,-0.000028,0.000026,-0.000037,0.0,-0.785398,1.691405e+09,DEV_DET_LOGGING
1,DEV_DET_LOGGING,/home/patrick/github/autolamella/autolamella/l...,1.691405e+09,None,2,02-hardy-panda,/home/patrick/github/autolamella/autolamella/l...,-0.000121,0.000031,-0.000018,0.0,-0.785398,1.691405e+09,DEV_DET_LOGGING


In [ ]:


# plot scatter with plotly
import plotly.express as px

fig = px.scatter(df_det, x="dpx_x", y="dpx_y", 
    color="stage", symbol="feature", hover_data=["lamella", "step", "is_correct"])

fig.show()

In [ ]:
# calculate percentage correct per feature

df_group = df_det.groupby(["feature", "is_correct"]).count().reset_index() 

df_group = df_group.pivot(index="feature", columns="is_correct", values="lamella")
display(df_group)

df_group["total"] = df_group["True"] + df_group["False"]
df_group["percent_correct"] = df_group["True"] / df_group["total"]
df_group["percent_correct"] = df_group["percent_correct"].round(2)
df_group = df_group.sort_values(by="percent_correct", ascending=False)
df_group.reset_index(inplace=True)
display(df_group)
